In [62]:
COLORS = {
    'reset': '\033[0m',

    'red': '\033[31m',
    'green': '\033[32m',
    'yellow': '\033[33m',
}

def GetColor(color_name: str) -> str:
    return COLORS.get(color_name, COLORS['reset'])

def Colored(text: str, color_name: str = 'green') -> str:
    return f"{GetColor(color_name)}{text}{GetColor('reset')}"


In [ ]:
import numpy as np

def MakeVectors(*args) -> np.ndarray:
    return np.array(object=args)

def DotProduct(matrix: np.ndarray, vector: np.ndarray) -> np.ndarray:
    return np.dot(matrix, vector)

# =============================================================================

def TransitionMatrix(matrix: list) -> np.ndarray:
    return MakeVectors(*matrix)

def PeopleVector(people_industry: int, people_academia: int) -> np.ndarray:
    return MakeVectors(people_industry, people_academia)

# =============================================================================

def CalculatePeopleDistribution(percent: float, total_people: int) -> int:
    return percent * total_people, (1 - percent) * total_people

def CalculatePeopleInitialState(
    total_people: int, percentage_industry: float = None,  percentage_academia: float = None, 
) -> np.ndarray:

    if percentage_industry is not None:
        people_industry, people_academia = CalculatePeopleDistribution(percentage_industry, total_people)
        return PeopleVector(people_industry, people_academia)

    elif percentage_academia is not None:
        people_academia, people_industry = CalculatePeopleDistribution(percentage_academia, total_people)
        return PeopleVector(people_industry, people_academia)
    
    else:
        raise ValueError("Either percentage_industry or percentage_academia must be provided.")

def CalculatePeopleNextYear(transition_matrix: np.ndarray, people: np.ndarray) -> np.ndarray:
    return DotProduct(transition_matrix, people)

# =============================================================================

def SimulateYears(
    initial_year: int, period_of_time: int,
    people: np.ndarray, transition_matrix: np.ndarray, step: int = 1,
) -> np.ndarray:
    # NOTE:
    # 1. people: np.ndarray = [people_industry, people_academia]
    # 2. transition_matrix: np.ndarray = [[to_industry_from_industry, to_industry_from_academia],
    #                                     [to_academia_from_industry, to_academia_from_academia]]
    history = []
    
    start = initial_year
    stop = initial_year + period_of_time
    
    history.append([start, people[0], people[1]])

    for year in range(start + 1, stop + 1, step):
        people = CalculatePeopleNextYear(transition_matrix, people)
        history.append([year, people[0], people[1]])

    return np.array(history)

# =============================================================================

def UnpackData(history: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    X_axis = history[:, 0]
    Y_axis_industry = history[:, 1]
    Y_axis_academia = history[:, 2]
    return X_axis, Y_axis_industry, Y_axis_academia

# =============================================================================

def FindChangeYears(people_history: list[tuple[int, np.ndarray]]) -> list[int]:

    # Unpack the data into separate arrays for years, industry counts, and academia counts:
    X_axis, Y_axis_industry, Y_axis_academia = UnpackData(people_history)

    # Determine the leader for each year. 
    # 1 for industry, -1 for academia, 0 for equal.
    leader_array = np.sign(Y_axis_industry - Y_axis_academia)
    
    # Find the indices where the leader changes.
    # We compare each element to the previous one.
    change_indices = np.where(leader_array[:-1] != leader_array[1:])[0]
    
    # The change happens at the *next* year, so we add 1 to the indices.
    # We then use these indices to get the corresponding years.
    change_years = X_axis[change_indices + 1]
    
    return change_years.tolist()



In [64]:
def GetTicks(initial_value: float, final_value: float, num_ticks: int = 21, dtype: object = int) -> np.ndarray:
    return np.linspace(initial_value, final_value, num_ticks, dtype=dtype)

def GetMinMaxValues(*args: list[float], delta_up: float = 1.1, delta_down: float = 0.8) -> tuple[float, float]:
    return min(min(arg) for arg in args) * delta_down, max(max(arg) for arg in args) * delta_up

def GetGraphicParameters(
    people_history: list[tuple[int, np.ndarray]],
    initial_year: int, period_of_time: int,
) -> dict[str, object]:
    
    X_axis, Y_axis_industry, Y_axis_academia = UnpackData(people_history)
    Y_min, Y_max = GetMinMaxValues(Y_axis_industry, Y_axis_academia)
    
    return X_axis, Y_axis_industry, Y_axis_academia, {
        'Y_min': Y_min, 'Y_max': Y_max,
        'X_ticks': GetTicks(initial_value = initial_year, final_value = initial_year + period_of_time), 
        'Y_ticks': GetTicks(initial_value=Y_min, final_value=Y_max, dtype=float), 
    }


## `Absolute Values`: 

In [70]:
T = [
    [0.99, 0.2], 
    [0.01, 0.8],
]

transitionMatrix = TransitionMatrix(matrix=T)

## `Initial Values`:


In [66]:
totalPeople = 1_000
percentagePeopleIndustry = 0.1

initialYear = 2025
periodOfTime = 100
finalYear = initialYear + periodOfTime



In [76]:
def RawSimulation(
    
    total_people: int,
    percentage_people_industry: float,
    
    initial_year: int,
    period_of_time: int,
    transition_matrix: np.ndarray,

) -> tuple[np.ndarray, list[int], list[int]]:
    
    people = CalculatePeopleInitialState(
        total_people=total_people, 
        percentage_industry=percentage_people_industry,
    )

    peopleHistory = SimulateYears(
        people=people,
        initial_year=initial_year,
        period_of_time=period_of_time,
        transition_matrix=transition_matrix,
    )

    change_years = FindChangeYears(people_history=peopleHistory)
    difference_change_years = [abs(initial_year - year) for year in change_years]

    return peopleHistory, change_years, difference_change_years




In [ ]:
# def Simulation(
#     total_people: int,
#     percentage_people_industry: float,
    
#     initial_year: int,
#     period_of_time: int,
#     transition_matrix: np.ndarray,

#     qnt_sample_total_people: int = 5,
#     step_sample_total_people: int = 10_000,

#     qnt_sample_percentage_people_industry: int = 5,
#     step_sample_percentage_people_industry: float = 0.1,
    
# ) -> tuple[np.ndarray, list[int], list[int]]:

#     Results = []
#     for i in range(qnt_sample_total_people):
#         total_people_i = total_people + i * step_sample_total_people

#         for j in range(qnt_sample_percentage_people_industry):
#             percentage_people_industry_j = percentage_people_industry + j * step_sample_percentage_people_industry

#             peopleHistory, change_years, difference_change_years = RawSimulation(
                
#                 total_people=total_people_i,
#                 percentage_people_industry=percentage_people_industry_j,
                
#                 initial_year=initial_year,
#                 period_of_time=period_of_time,
#                 transition_matrix=transition_matrix,
#             )

#             Results.append((total_people_i, percentage_people_industry_j, peopleHistory, change_years, difference_change_years))
    
#     return Results


In [ ]:

# def Simulation(
#     total_people: int,
#     percentage_people_industry: float,
#     initial_year: int,
#     period_of_time: int,
#     transition_matrix: np.ndarray,
#     qnt_sample_total_people: int = 5,
#     step_sample_total_people: int = 10_000,
#     qnt_sample_percentage_people_industry: int = 5,
#     step_sample_percentage_people_industry: float = 0.1,
# ) -> np.ndarray:
    
#     # Calculate the number of years in the simulation
#     num_years = period_of_time + 1
    
#     # Initialize a 4D tensor with zeros to store all results
#     # Shape: (total_people_samples, percentage_samples, years, data_points)
#     all_results_tensor = np.zeros(
#         (qnt_sample_total_people, qnt_sample_percentage_people_industry, num_years, 3)
#     )

#     for i in range(qnt_sample_total_people):
#         total_people_i = total_people + i * step_sample_total_people

#         for j in range(qnt_sample_percentage_people_industry):
#             percentage_people_industry_j = percentage_people_industry + j * step_sample_percentage_people_industry

#             people = CalculatePeopleInitialState(
#                 total_people=total_people_i, 
#                 percentage_industry=percentage_people_industry_j,
#             )

#             peopleHistory = SimulateYears(
#                 people=people,
#                 initial_year=initial_year,
#                 period_of_time=period_of_time,
#                 transition_matrix=transition_matrix,
#             )

#             # Assign the peopleHistory array directly to the tensor slice
#             all_results_tensor[i, j, :, :] = peopleHistory
    
#     return all_results_tensor

In [95]:
def Simulation(
    total_people: int,
    percentage_people_industry: float,
    initial_year: int,
    period_of_time: int,
    transition_matrix: np.ndarray,
    qnt_sample_total_people: int = 5,
    step_sample_total_people: int = 10_000,
    qnt_sample_percentage_people_industry: int = 5,
    step_sample_percentage_people_industry: float = 0.1,
) -> dict[str, np.ndarray]:
    
    # Calculate dimensions for our tensors
    num_years = period_of_time + 1
    total_people_samples = qnt_sample_total_people
    percentage_samples = qnt_sample_percentage_people_industry

    # Initialize tensors to store the results
    people_history_tensor = np.zeros(
        (total_people_samples, percentage_samples, num_years, 3)
    )
    change_years_tensor = np.empty((total_people_samples, percentage_samples), dtype=object)
    difference_years_tensor = np.empty((total_people_samples, percentage_samples), dtype=object)

    for i in range(total_people_samples):
        total_people_i = total_people + i * step_sample_total_people

        for j in range(percentage_samples):
            percentage_people_industry_j = percentage_people_industry + j * step_sample_percentage_people_industry

            # Run a single raw simulation
            peopleHistory, change_years, difference_change_years = RawSimulation(
                total_people=total_people_i,
                percentage_people_industry=percentage_people_industry_j,
                initial_year=initial_year,
                period_of_time=period_of_time,
                transition_matrix=transition_matrix,
            )

            # Store the results in the appropriate tensors
            people_history_tensor[i, j, :, :] = peopleHistory
            change_years_tensor[i, j] = change_years
            difference_years_tensor[i, j] = difference_change_years
            
    # Return a dictionary containing all the tensors
    return {
        'people_history': people_history_tensor,
        'change_years': change_years_tensor,
        'difference_change_years': difference_years_tensor,
    }

In [99]:

Responses = Simulation(
    total_people = totalPeople,
    initial_year = initialYear,
    period_of_time = periodOfTime,
    transition_matrix = transitionMatrix,
    percentage_people_industry = percentagePeopleIndustry,
)

Responses['people_history'][0]

array([[[2025.        ,  100.        ,  900.        ],
        [2026.        ,  279.        ,  721.        ],
        [2027.        ,  420.41      ,  579.59      ],
        ...,
        [2123.        ,  952.3809523 ,   47.6190477 ],
        [2124.        ,  952.38095232,   47.61904768],
        [2125.        ,  952.38095233,   47.61904767]],

       [[2025.        ,  200.        ,  800.        ],
        [2026.        ,  358.        ,  642.        ],
        [2027.        ,  482.82      ,  517.18      ],
        ...,
        [2123.        ,  952.38095231,   47.61904769],
        [2124.        ,  952.38095233,   47.61904767],
        [2125.        ,  952.38095234,   47.61904766]],

       [[2025.        ,  300.        ,  700.        ],
        [2026.        ,  437.        ,  563.        ],
        [2027.        ,  545.23      ,  454.77      ],
        ...,
        [2123.        ,  952.38095232,   47.61904768],
        [2124.        ,  952.38095233,   47.61904767],
        [2125.        

In [ ]:
A = [
    [[people1, percentage1], [people1, percentage2], [people1, percentage3]],
    [[people2, percentage1], [people2, percentage2], [people2, percentage3]],
    [[people3, percentage1], [people3, percentage2], [people3, percentage3]],
]

